In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,"../") 
from codes.functions.check_policy import load_info
import numpy as np
import qutip as qt
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, inset_axes, InsetPosition, BboxPatch, BboxConnector
import seaborn as sns
import matplotlib.gridspec as gridspec
#matplotlib.use('module://ipykernel.pylab.backend_inline')
from cycler import cycler
import matplotlib.font_manager as fm
import matplotlib.patheffects as path_effects
import matplotlib
from cycler import cycler
from IPython.display import SVG 
import scipy
import pandas as pd



plt.style.use('seaborn') 
fontsize = 12

mpl_settings = {
    'text.usetex': False,
    'font.family': 'sans-serif',
    'pgf.texsystem': 'pdflatex',
    'font.size': fontsize,
    
    'axes.labelsize': fontsize,               # -> axis labels
    'legend.fontsize': fontsize,              # -> legends
    'xtick.labelsize': fontsize,
    'ytick.labelsize': fontsize,
    'ytick.major.size': 3,
    'xtick.major.size': 3,
    'axes.linewidth': 2,
    'axes.edgecolor': "0.15",
    'figure.dpi': 300
}


plt.rcParams.update(mpl_settings)

/u/pori/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/u/pori/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/u/pori/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/u/pori/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/u/pori/conda-envs/conda_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
from matplotlib.font_manager import findfont, FontProperties
font = findfont(FontProperties(family=['sans-serif']))
font

'/zeropoint/u/pori/conda-envs/conda_env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'

# Figure 6

In [3]:
a = np.loadtxt("training/2020-10_optimal_alpha/23_10_20_optimal_analytical_solution_out_noise_v11.txt")

appo = np.zeros((len(a), 3))
appo[:,:2] = a[:,:2]
appo[:, 2] = a[:,2:].mean(axis=1)

df = pd.DataFrame(appo, columns=["Decay","Dephasing","Prob_fin"])
DF = df.apply(pd.to_numeric)
df=DF.reset_index(drop=True)
df=df.drop_duplicates(subset=['Decay', 'Dephasing'])
df_analytical=df.pivot('Decay', 'Dephasing', 'Prob_fin')


folders=["training/2021-05-25_noise_last_v2/summaries",
         "training/2021-05-26_noise_last_v2/summaries",
         "training/2021-05-27_noise_last_v2/summaries"]
count=0
pairs=[]
probs_fin=[]
for folder in folders:
    for filename in os.listdir(folder):
        if filename.endswith('.npy'):
            split=(filename.split("_"))
            decay=float(split[4])
            dephasing=float(split[5].split(".npy")[0])
            summary = np.load(folder+"/"+filename, allow_pickle=True).item()
            pairs.append([decay,dephasing])
            probs_fin.append(float(np.max(summary["probs_final"] ) ))

            count+=1

DF=pd.DataFrame(pairs,columns=["decay","dephasing"])

DF["probs_fin"]=np.array(probs_fin)
DF = DF.apply(pd.to_numeric)
df=DF


df=df.drop_duplicates(subset=["decay","dephasing"])


df_RL=df.pivot("decay","dephasing", 'probs_fin')


X=df_analytical.columns.values
Y=df_analytical.index.values
Z_analytical=df_analytical.values

Z_RL=df_RL.values
Z_RL=df_RL.values


In [4]:
scale=1.5
fig = plt.figure(figsize=(4*scale, 3*scale), dpi=300)
main_grid = fig.add_gridspec(1, 1, wspace=0.3)
gs1 = main_grid[0].subgridspec(2, 1, hspace=0.3)


mpl_settings = {
    'grid.linestyle': "dashed",
    'grid.color': "gray",
    'grid.alpha': 0.6
}
plt.rcParams.update(mpl_settings)

extent=(0, 1, 0, Y[-1]/20)
ax1= fig.add_subplot(gs1[0])
ax2= fig.add_subplot(gs1[1])
# ax1.imshow(Z_analytical,
#              origin="lower",
#              extent=extent,
#              vmin=0,
#              vmax=1,
#              aspect="auto",
#              cmap="YlOrRd") 

ax1.set_ylim(0, 1.1)

ax1.set_ylabel(r"<$F(T_{\rm max})>$")
w=6
#y = np.convolve(dephasing_after[:, :, -1].mean(axis=1), np.ones(w), 'valid') / w
y = Z_RL[0, :]
ax1.plot(np.linspace(0, 1, len(y)), y, label="Dephasing")
ax1.hlines(max(y), 0, 40, linestyle="dashed", color="k")
w=1
#y = (np.convolve(decay_after[:, :, -1].mean(axis=1), np.ones(w), 'valid') / w)[:40]
y = Z_RL[:, 0]
ax1.plot(np.linspace(0,1, len(y)), y, label="Decay")

ax1.set_xlim(0, 1)
#ax[1].plot(decay[:40])
#ax[1].set_ylim(0, 1)
ax1.legend()

def deg2rad(x):
    return x /40

def rad2deg(x):
    return x *40

secax = ax1.secondary_xaxis('top', functions=(deg2rad, rad2deg))
secax.set_xlabel(r'$\gamma_{decay}/\gamma_{meas}$', labelpad=10)
secax.xaxis.set_tick_params(pad=5)





im = ax2.imshow(Z_RL,
                origin="lower",
                extent=extent,vmin=0.7,
                vmax=1,aspect="auto",
                cmap = matplotlib.cm.get_cmap('YlOrRd_r')
               )


#ax[1].set_ylabel(r"$T_{max}*max(\alpha)$")
ax1.set_title("Greedy strategy")
# ax1.set_ylabel(r"$\gamma_{decay}$")
ax2.set_ylabel(r"$\gamma_{decay}/\gamma_{meas}$")
ax2.set_xlabel(r"$\gamma_{depashing}/\gamma_{meas}$")
ax2.yaxis.set(ticks=[0,Y[-1]/2/20,Y[-1]/20], ticklabels=[0, 0.0125, 0.025])
# ax1.set(yticks=[0,5,10,15])
ax1.axes.xaxis.set_ticklabels([])



cbar_ax = fig.add_axes([0.95, 0.1, 0.02, 0.8])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.ax.set_title("Fidelity", pad=20)
#cbar.ax.invert_yaxis()
fonts = 21
fig.text(-0.05, 0.93, "a",  fontweight='bold', fontsize=fonts)
fig.text(-0.05, 0.46, "b",  fontweight='bold', fontsize=fonts)
#ax[1].set_ylabel(r"$Average final fidelity$")
ax2.yaxis.set(ticklabels=["0.0000", "0.0125", "0.0250"])






fig.savefig("figure6.pdf", format = "pdf", dpi=300, bbox_inches='tight', pad_inches=0)
